In [30]:
import pandas as pd
import ast
import numpy as np
from pathlib import Path  
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import geopandas as gpd
import pandas as pd
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import seaborn as sns
import matplotlib.pyplot as plt
from adjustText import adjust_text
from matplotlib.patches import Patch
from sklearn.decomposition import PCA, FactorAnalysis
import matplotlib.colors as mcolors

import os
# os.environ["OMP_NUM_THREADS"] = "2"

from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_percentage_error, mean_squared_error

#import plotly.express as px
import urllib.request
import zipfile
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
from matplotlib.ticker import FuncFormatter
from sklearn.decomposition import PCA, FactorAnalysis
from factor_analyzer import FactorAnalyzer, calculate_kmo, calculate_bartlett_sphericity
from sklearn.preprocessing import StandardScaler
from factor_analyzer import FactorAnalyzer


import warnings
warnings.filterwarnings("ignore", message="KMeans is known to have a memory leak")


# Load the Excel file
df = pd.read_csv('../../data/raw/msa-municipality.csv')
averageT = pd.read_csv('../../data/raw/average_temperature.csv', skiprows=4, on_bad_lines='skip')
precipitation = pd.read_csv('../../data/raw/precipitation.csv', skiprows=4, on_bad_lines='skip')
palmerZ = pd.read_csv('../../data/raw/palmerzindex.csv', skiprows=3, on_bad_lines='skip')
# Create a new dataframe
data = []

# Iterate through each row in the original dataframe
for index, row in df.iterrows():
    msa = row.iloc[0]  # Get the MSA
    counties = ast.literal_eval(row.iloc[1])  # Convert the string representation of the list back to a list
    
    # Append each county with its corresponding MSA to the new data list
    for county in counties:
        data.append([county, msa])

# Create a new dataframe from the data list
msa_county = pd.DataFrame(data, columns=['County', 'MSA'])

# Separate county and state
msa_county [['County Name', 'State']] = msa_county ['County'].str.split(', ', expand=True)
msa_county  = msa_county[['County Name', 'State', 'MSA']]

# Remove the word "County" from the "County Name" column
msa_county['County Name'] = msa_county['County Name'].str.replace('County', '').str.strip()


msa_county

,County Name,State,MSA
0,Callahan,TX,"Abilene, TX"
1,Jones,TX,"Abilene, TX"
2,Taylor,TX,"Abilene, TX"
3,Portage,OH,"Akron, OH"
4,Summit,OH,"Akron, OH"
...,...,...,...
1176,Trumbull,OH,"Youngstown-Warren-Boardman, OH-PA"
1177,Mercer,PA,"Youngstown-Warren-Boardman, OH-PA"
1178,Sutter,CA,"Yuba City, CA"
1179,Yuba,CA,"Yuba City, CA"


In [31]:
# Dictionary to map full state names to state abbreviations
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', ' D.C.': 'DC'
}

# Step 1: Clean the first dataframe by removing 'County' from the 'Name' column and converting state names to abbreviations
averageT['Name'] = averageT['Name'].str.replace(' County', '', regex=False)
averageT['State'] = averageT['State'].apply(lambda x: state_abbreviations.get(x, x))

In [32]:
# Dictionary to map full state names to state abbreviations
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', ' D.C.': 'DC'
}

# Step 1: Clean the first dataframe by removing 'County' from the 'Name' column and converting state names to abbreviations
precipitation['Name'] = precipitation['Name'].str.replace(' County', '', regex=False)
precipitation['State'] = precipitation['State'].apply(lambda x: state_abbreviations.get(x, x))

In [33]:
# Dictionary to map full state names to state abbreviations
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', ' D.C.': 'DC'
}

# Step 1: Clean the first dataframe by removing 'County' from the 'Name' column and converting state names to abbreviations
palmerZ['Name'] = palmerZ['Name'].str.replace(' County', '', regex=False)
palmerZ['State'] = palmerZ['State'].apply(lambda x: state_abbreviations.get(x, x))

In [34]:
merged_data_1 = pd.merge(averageT, msa_county, left_on=['Name', 'State'], right_on=['County Name', 'State'], how='left')

# Assuming your DataFrame is named df
merged_data_1 = merged_data_1.dropna(subset=['MSA'])

merged_data_1['Value'] = pd.to_numeric(merged_data_1['Value'])

#take the average value of the summation of heating degree days
result = merged_data_1.groupby('MSA')['Value'].sum() / merged_data_1.groupby('MSA')['County Name'].count()
result = result.reset_index()
result.columns = ['MSA', 'Average Temperature (F)']
result

,MSA,Average Temperature (F)
0,"Abilene, TX",65.000000
1,"Akron, OH",52.500000
2,"Albany, GA",67.725000
3,"Albany-Lebanon, OR",50.200000
4,"Albany-Schenectady-Troy, NY",48.860000
...,...,...
364,"Yakima, WA",47.900000
365,"York-Hanover, PA",55.000000
366,"Youngstown-Warren-Boardman, OH-PA",51.033333
367,"Yuba City, CA",63.950000


In [35]:
merged_data_2 = pd.merge(precipitation, msa_county, left_on=['Name', 'State'], right_on=['County Name', 'State'], how='left')

# Assuming your DataFrame is named df
merged_data_2 = merged_data_2.dropna(subset=['MSA'])

merged_data_2['Value'] = pd.to_numeric(merged_data_2['Value'])


#take the average value of the summation of heating degree days
result_2 = merged_data_2.groupby('MSA')['Value'].sum() / merged_data_2.groupby('MSA')['County Name'].count()
result_2 = result_2.reset_index()
result_2.columns = ['MSA', 'Precipitation (inches)']
result_2

,MSA,Precipitation (inches)
0,"Abilene, TX",27.356667
1,"Akron, OH",40.550000
2,"Albany, GA",58.845000
3,"Albany-Lebanon, OR",65.670000
4,"Albany-Schenectady-Troy, NY",37.660000
...,...,...
364,"Yakima, WA",22.810000
365,"York-Hanover, PA",42.820000
366,"Youngstown-Warren-Boardman, OH-PA",45.596667
367,"Yuba City, CA",12.295000


In [36]:
merged_data_3 = pd.merge(palmerZ, msa_county, left_on=['Name', 'State'], right_on=['County Name', 'State'], how='left')

# Assuming your DataFrame is named df
merged_data_3 = merged_data_3.dropna(subset=['MSA'])

merged_data_3['Value'] = pd.to_numeric(merged_data_3['Value'])

#take the average value of the summation of heating degree days
result_1 = merged_data_3.groupby('MSA')['Value'].sum() / merged_data_3.groupby('MSA')['County Name'].count()
result_1 = result_1.reset_index()
result_1.columns = ['MSA', 'Palmer Z Index']
result_1

,MSA,Palmer Z Index
0,"Abilene, TX",0.340000
1,"Akron, OH",0.165000
2,"Albany, GA",0.547500
3,"Albany-Lebanon, OR",-0.350000
4,"Albany-Schenectady-Troy, NY",-0.396000
...,...,...
364,"Yakima, WA",-0.860000
365,"York-Hanover, PA",-0.220000
366,"Youngstown-Warren-Boardman, OH-PA",0.783333
367,"Yuba City, CA",-1.580000


In [37]:
merged_data = result_1.merge(result, on="MSA")
merged_data = merged_data.merge(result_2, on="MSA")
merged_data

,MSA,Palmer Z Index,Average Temperature (F),Precipitation (inches)
0,"Abilene, TX",0.340000,65.000000,27.356667
1,"Akron, OH",0.165000,52.500000,40.550000
2,"Albany, GA",0.547500,67.725000,58.845000
3,"Albany-Lebanon, OR",-0.350000,50.200000,65.670000
4,"Albany-Schenectady-Troy, NY",-0.396000,48.860000,37.660000
...,...,...,...,...
364,"Yakima, WA",-0.860000,47.900000,22.810000
365,"York-Hanover, PA",-0.220000,55.000000,42.820000
366,"Youngstown-Warren-Boardman, OH-PA",0.783333,51.033333,45.596667
367,"Yuba City, CA",-1.580000,63.950000,12.295000


In [38]:
merged_data.to_csv('../../data/tidy/other-climate-data.csv', index=False)